## Compute Graph Manually (Task 1 And Task 2)

In [28]:
from graphviz import Digraph # to view graph
import numpy as np

########################################## STEP 1: CREATE NODES AND CHILDREN ##################################################
class Node:
    #Creates Node,its children, its operation and connections
   
    def __init__(self, name, op, parents=None):
        self.name = name
        self.op = op
        self.parents = [] if parents is None else parents #to keep parents of node
        self.children = [] #to keep children if any
    def inputs(self):
        return self.parents
    def consumers(self):
        return self.children

def children(nodes):
    #takes node and add as children to the parent node
    for node in nodes:
        if node.parents is not None:
            for p in node.parents:
                p.children.append(node)

def sortNodes(nodes):
    #takes the list of nodes of graph and does sorting for backpropogation 
    #and returns sorted graph node
    # start by collecting input nodes,
    parameter_nodes = [n for n in nodes if len(n.parents)==0]
    #check if any input nodes
    assert(len(parameter_nodes) > 0), 'no input nodes found'
    # get nodes, and sort them
    unsorted_internal_nodes = [n for n in nodes if n not in parameter_nodes]
    internal_nodes = []
    # iteratively remove sinks to perform topological sort
    while len(unsorted_internal_nodes) > 0:
        for n in unsorted_internal_nodes:
            # loop through unsorted nodes and check if it has children and is a sink node or not
            is_sink = True
            for c in n.children:
                if c in unsorted_internal_nodes:
                    is_sink = False
                    break
            if is_sink:
                unsorted_internal_nodes.remove(n)
                internal_nodes.append(n)
    # reversing internal nodes as first nodes in the list should be the one that'll be evaluated first
    return parameter_nodes, internal_nodes[::-1]

########################################## STEP 2: FORWARD PROPOGATION #####################################################
def forwardprop(nodes, parameter_table):
    #does forward propagation, takes nodes and
    # return the activation corresponding to the output node and 
    #table storing activations for each node in the graph
    
    # assign inputs values
    for name, tensor in parameter_table.items():
        nodes[name].op.assign(tensor)
    # propagate, add activations in a dictionary
    output_table = {}
    for name, node in nodes.items():
        # collect parents activations
        args = [output_table[p.name] for p in node.inputs()]
        output_table[node.name] = node.op.forward(*args)
   
    return output_table[list(nodes.keys())[-1]], output_table

########################################## STEP 3: COMPUTE GRADIENTS #####################################################
def computeGrad(node, output_table, parameter_grad_table, output_grad_table):
    #Computes gradient for each node and returns gradient
    #if already computed return the table
    if node.name in output_grad_table:
        return output_grad_table[node.name]
    # list of gradients 
    consumer_gradients = [] 
    for consumer_node in node.consumers():
        # index of node in its consumer node inputs
        parameter_index = consumer_node.inputs().index(node)
        #check for nodes in parameter_grad_table that stores gradients at nodes input, 
        #which may be summed to compute gradients at node outputs
        if consumer_node.name in parameter_grad_table:
            consumer_gradients.append(parameter_grad_table[consumer_node.name][parameter_index])
        else:
            consumer_parameter_activations = [output_table[n.name] for n in consumer_node.inputs()]
            # compute the gradient at consumer node output by calling recursively
            consumer_output_gradient = computeGrad(consumer_node, output_table, parameter_grad_table, output_grad_table)
            # based on gradient at consumer node output, compute gradients at its inputs
            consumer_parameter_gradients = consumer_node.op.backward(\
                consumer_output_gradient, *consumer_parameter_activations)
            # add gradients at consumer node inputs
            parameter_grad_table[consumer_node.name] = consumer_parameter_gradients
           
            #add to dictionary
            consumer_gradients.append(consumer_parameter_gradients[parameter_index])
    # gradient at node output is the sum of the gradients at its consumer's inputs
    consumer_gradients_sum = reduce((lambda x, y: x + y), consumer_gradients)
    # cache gradient at node output
    output_grad_table[node.name] = consumer_gradients_sum
    return consumer_gradients_sum
########################################## STEP 4: BACKWARD PROPOGATION #####################################################
def backprop(nodes, output_table):
    #does backpropogation and takes output_table which is used to store outputs computed during the forward pass
    # and returns table that stores gradients for 
    # stores gradients at nodes output
    output_grad_table = {}
    # stores gradients at nodes input, they may be summed to compute gradients at node outputs
    parameter_grad_table = {} 

    # gradient at all sinks node is one
    sinks = [n for n in nodes if len(n.consumers()) == 0]
    for n in sinks:
        output_grad_table[n.name] = np.ones(output_table[n.name].shape)
    #getting gradient of rest of the nodes
    for node in nodes:
        computeGrad(node, output_table, parameter_grad_table, output_grad_table)

    return output_grad_table

####################################### STEP 5: CREATE GRAPH AND CALL NODE CLASS ##############################################
class Graph:

    def __init__(self, name):
        self.name = name
        #dictionary of all nodes
        self.nodes = {}
        self.parameter_nodes = None
        # forward propagation table values
        self.output_table = None

    def add(self, op, name, inputs=None):
        # to make sure name is unique
        if name in self.nodes:
            raise Exception('node name {} already used'.format(name))
        # checking if number of inputs provided was right
        if not (inputs is None and op.num_inputs() == 0 \
            or inputs is not None and op.num_inputs() == len(inputs)):
            raise Exception('Inputs do not match, expected {}, received {}'\
                .format(str(op.num_inputs()), str(0 if inputs is None else len(inputs))))

        input_nodes = None
        if inputs is not None:
            # collect parents based on their name if strings were passed
            if len(inputs) > 0 and type(inputs[0]) == str:
                input_nodes = [self.nodes[n] for n in inputs]
            else:
                input_nodes = inputs
        node = Node(name, op, input_nodes)
        self.nodes[name] = node
        return node

    def completeGraph(self):
        #this function is called once all nodes are added to the graph 
        #and set the connection of children based on parents and perform the sorting for backpropogation 
        #calling to add children to parent
        children(self.nodes.values())
        #sorting all nodes
        self.parameter_nodes, internal_nodes = sortNodes(self.nodes.values())
        # reordering nodes dictionary
        self.nodes = {n.name: n for n in self.parameter_nodes}
        for n in internal_nodes:
            self.nodes[n.name] = n
            
####################################### STEP 6: VISUALIZATION OF GRAPH ##############################################
    def visualize(self):
        #visualizing the graph using GraphViz library
        #check if all nodes are set and sorted
        assert(self.parameter_nodes is not None), 'Visualization failed.'
        g = Digraph('G', filename='graph/'+self.name, format='png')
        filepath='graph/'+self.name
        #creating node and edges on graph
        for node in self.nodes.values():
            node_str = node.name if self.output_table is None else '{}, {}'\
                .format(node.name, str(self.output_table[node.name].shape))
            g.node(node.name, node_str)
        for node in self.nodes.values():
            for p in node.parents:
                g.edge(p.name, node.name)
        #opens default photo viewer to view graph
        g.view()
        return ('graph drawn successfully at: ' + filepath)

    def forward(self, parameter_table):
        # make sure a value has been provided for each input
        for i in self.parameter_nodes:
            if i.name not in parameter_table:
                raise Exception('missing value for parameter {}'.format(i.name))
        #does forward propogation and storing in output_table for using it later in backpropogation
        output, self.output_table = forwardprop(self.nodes, parameter_table)
        return output, self.output_table

    def backward(self):
        #check if output_table is None, if None that means no forward propogation is done and 
        #hence no back propogation is performed
        if self.output_table is None:
            raise Exception('no output_table, \
                you may be attempting to backprop without having first performed forward propagation')
        return backprop(list(self.nodes.values()), self.output_table)

####################################### STEP 7: DEFINE OPERATION ON NODES ##############################################
class OpValue:
    #to assign values to node in the graph
    def num_inputs(self):
        return 0
    def assign(self, X):
        self.X = X
    def forward(self):
        return self.X

#for addition 
class OpAdd:
    def num_inputs(self):
        return 2
    def forward(self, A, B):
        return A + B
    #assigning gradients
    def backward(self, grad, A, B):
        dA = grad
        dB = grad
        return [dA, dB]
#multiplication of two inputs
class OpMultiply:
    def num_inputs(self):
        return 2
    def forward(self, A, B):
        return np.dot(A, B)
    def backward(self, grad, A, B):
        dA = np.dot(grad, B.T)
        dB = np.dot(A.T, grad)
        return [dA, dB]


########################################### STEP 8: IMPLEMENTATION ######################################################
#create graph
g = Graph('my_graph')

#create nodes for loss function f=w0^3 *w1 + w0*w1 +2
W_0 = g.add(OpValue(), 'W_0') # weight 1
W_1 = g.add(OpValue(), 'W_1') # weight 2
b_2 = g.add(OpValue(), 'b_2') # bias 2
#multiplying w0 thrice to get w0^3
Mul = g.add(OpMultiply(), 'Mul', [W_0, W_0])
Mul1 = g.add(OpMultiply(), 'Mul1', [Mul, W_0])
#ultiplying to get w0^3 *w1
Mul2 = g.add(OpMultiply(), 'Mul2', [Mul1, W_1])
#multiplying w0*w1
Mul3= g.add(OpMultiply(), 'Mul3', [W_0, W_1])
#doing Summation to get w0^3 *w1 + w0*w1 +2
Add= g.add(OpAdd(), 'Add', [Mul2, Mul3])
Add1= g.add(OpAdd(), 'Add1', [Add, W_1])
Add2= g.add(OpAdd(), 'Add2', [Add1, b_2])

#calling graph method to complete all nodes and sorting of nodes
g.completeGraph()
#visualizing graph and saving to specified filePath
result=g.visualize()
print(result)

# initialize inputs w0=2,w1=3 and constant 2 (b_2)
parameter_table = {
    'W_0': np.array([2]),
    'W_1': np.array([3]),
    'b_2': np.array([2])}
########################################### STEP 9: CALL FORWARD PROPOGATION ###################################################
# do forward propagation and calculate the loss function
output, output_table = g.forward(parameter_table)
print('Loss Function value when w0=2,w1=3 :' ,output)
########################################### STEP 10: CALL BACK PROPOGATION ####################################################
# do backpropogation to compute gradients at all levels
grad_table = g.backward()
print ('Gradient with respect to w0:', grad_table['W_0'])
print ('Gradient with respect to w1:', grad_table['W_1'][0])

graph drawn successfully at: graph/my_graph
Loss Function value when w0=2,w1=3 : [35]
Gradient with respect to w0: 39.0
Gradient with respect to w1: 11.0


## Compute Graph Using Tensorflow (Task 3)

In [29]:
import tensorflow as tf

tf.reset_default_graph()
################################ STEP 1: INITIALIZE VARIABLES AND CONSTANT ###################################################
#Compute loss function = (w0^3)*w1 + w0*w1 + w1 + 2
# Declaring two weights w0 and w1 with values 2, 3 and constant with value 2
w0 = tf.Variable(2, name='w0')
w1 = tf.Variable(3, name='w1')
c=tf.constant(2, name='const_2')
######################################## STEP 2: DEFINE LOSS FUNCTION  #######################################################
f=w0* w0*w0*w1 + w0*w1 + w1 + c
filepath='graph/'
################################ STEP 3: WRITE SUMMMARY FOR VISUALIZATION ###################################################
writer=tf.summary.FileWriter(filepath,tf.get_default_graph())
#################################### STEP 4: RUN THE SESSION ###############################################################
with tf.Session() as sess:
    #initializing both the variables
    sess.run(w0.initializer)
    sess.run(w1.initializer)
    #executing the loss function f defined earlier
    result=sess.run(f)
    print('Loss Function value when w0=2,w1=3 :' ,result)
#closing the writer that is used to write graph for visualizing using Tensorboard
writer.close()
print('graph written successfully at :' ,filepath)

Loss Function value when w0=2,w1=3 : 35
graph written successfully at : graph/


## Compute Gradient Using tf.gradients() (Task 4)

In [30]:
tf.reset_default_graph()
################################ STEP 1: INITIALIZE VARIABLES AND CONSTANT ###################################################
#Compute loss function = (w0^3)*w1 + w0*w1 + w1 + 2
# Declaring two weights w0 and w1 with values 2, 3 and constant with value 2
w0 = tf.Variable(2, name='w0')
w1 = tf.Variable(3, name='w1')
c=tf.constant(2, name='const_2')
######################################## STEP 2: DEFINE LOSS FUNCTION  #######################################################
f=w0* w0*w0*w1 + w0*w1 + w1 + c
######################################## STEP 3: COMPUTE GRADIENTS  #######################################################
#get gradient w.r.t w1
grad1 = tf.gradients(f, [w1])
#get gradient w.r.t w0
grad = tf.gradients(f, w0)

#################################### STEP 4: RUN THE SESSION ###############################################################
sess = tf.Session()
#initialize all the variables
sess.run(tf.global_variables_initializer())

#get gradient w.r.t w0
res = sess.run(grad)
#get gradient w.r.t w1
res1 = sess.run(grad1)
print('Gradient with respect to w0:' ,res) 
print('Gradient with respect to w1:',res1) 

Gradient with respect to w0: [39]
Gradient with respect to w1: [11]
